In [1]:
from time import sleep

from qcodes.station import Station
from Baspi_Lnhrdac2 import BaspiLnhrdac2

In [2]:
station = Station()
DAC = BaspiLnhrdac2('LNHRDAC', 'TCPIP0::192.168.0.5::23::SOCKET')
station.add_component(DAC)


Connected to: Basel Precision Instruments GmbH (BASPI) LNHR DAC II (SP1060) - 24 channel version (serial:SN 10600000011, firmware:Revision 3.4.9u) in 6.48s
All channels have been turned off (1 MOhm Pull-Down to AGND) upon initialization and are pre-set to 0.0 V if turned on without setting a voltage beforehand.



'LNHRDAC'

In [3]:
DAC.ch14.voltage.set(5.86)
DAC.ch14.high_bandwidth.set("off")
DAC.ch14.enable.set("on")

voltage = DAC.ch14.voltage.get()
print(f"{voltage} V")

5.86 V


In [12]:
configuration = {
    "shape": "sine",
    "frequency": 100,
    "amplitude": 3.4,
    "offset": -0.4,
    "phase": 0
}

DAC.swg.awg.set("A")
DAC.swg.configuration.set(configuration)
DAC.swg.apply()

In [5]:
DAC.awga.channel.set(8)
DAC.awga.cycles.set(1000)
DAC.awga.trigger.set("disable")
DAC.ch8.enable.set("on")
DAC.awga.enable.set(True)